In [6]:
from math import sqrt

In [7]:
def line(x):
    return lambda w,b: w*x + b       

In [8]:
f = line(3)
print(f(3, 2))

11


In [9]:
# recursive rank function 
def ranked(t):
    if not isinstance(t, list):
        return 0
    return 1 + ranked(t[0])

In [10]:
t = [9, 16 , 25]
print(f"Rank of {t} = {ranked(t)}")

Rank of [9, 16, 25] = 1


In [11]:
#unwrapped rank function
def ranker(t, a):
    if not isinstance(t, list):
        return a
    else:
        return ranker(t[0], a+1)
    
def rank(t):
    return ranker(t, 0) 

In [12]:
print(f"Rank of {t} = {ranked(t)}")

Rank of [9, 16, 25] = 1


In [13]:
# sum1 function

def sum1(t):
    if rank(t) == 1 and len(t) > 0:
        return summer1(t, len(t)-1, 0)
    else:
        return -1

def summer1(t,i, a):
    if i == 0:
        return t[0] + a
    else:
        return summer1(t, i - 1, t[i] + a)

In [14]:
sum1(t)

50

In [15]:
# squareroot function 

def sqroot(t):
    return sqrooted(t)

def sqrooted(t):
    if isinstance(t, list):
        return [sqrooted(x) for x in t]
    else: return sqrt(t)

In [16]:
sqroot(t)

[3.0, 4.0, 5.0]

In [17]:
# square function 

def sqr(t):
    return sqred(t)

def sqred(t):
    if isinstance(t, list):
        return [sqred(x) for x in t]
    else: return  t * t

In [18]:
# pairwise sum

def hadamard_sum(a, b):
    if isinstance(a, list) and isinstance(b, list):
        return [hadamard_sum(x, y) for x, y in zip(a, b)]
    elif isinstance(a, list):
        return [hadamard_sum(x, b) for x in a]
    elif isinstance(b, list):
        return [hadamard_sum(a, y) for y in b]
    else:
        return a + b


def hadamard_sub(a, b):
    if isinstance(a, list) and isinstance(b, list):
        return [hadamard_sub(x, y) for x, y in zip(a, b)]
    elif isinstance(a, list):
        return [hadamard_sub(x, b) for x in a]
    elif isinstance(b, list):
        return [hadamard_sub(a, y) for y in b]
    else:
        return a - b


In [19]:
# pairwise product

def hadamard_product(a, b):
    if isinstance(a, list) and isinstance(b, list):
        return [hadamard_product(x, y) for x, y in zip(a, b)]
    elif isinstance(a, list):
        return [hadamard_product(x, b) for x in a]
    elif isinstance(b, list):
        return [hadamard_product(a, y) for y in b]
    else:
        return a * b

In [36]:
# Define a linear model: y = theta[0] * x + theta[1]
def line(xs):
    return lambda theta: [theta[0] * x + theta[1] for x in xs]

# Define a "plane" function: y = dot(theta[:-1], t) + theta[-1]
def plane(ts):  # ts is a list of input vectors: [[x1, x2], [x1, x2], ...]
    return lambda theta: [
        sum(t_i * w_i for t_i, w_i in zip(t, theta[:-1])) + theta[-1]
        for t in ts
    ]

# Define a second order quadratic model: y = theta[0] * x * x + theta[1] * x + theta[2]
def curve(xs):
   return lambda theta: [theta[0] * x * x + theta[1] * x + theta[2] for x in xs]


# Define the L2 loss function builder (squared error loss)
def l2_loss(model):
    def loss_on(xs, ys):
        def compute_loss(theta):
            pred_ys = model(xs)(theta)  # Predict outputs using current theta
            diffs = hadamard_sub(ys, pred_ys)  # Elementwise difference between actual and predicted
            return sum1(sqr(diffs))  # Sum of squared differences
        return compute_loss
    return loss_on

# Compute numerical gradient using finite differences
def numerical_gradient(loss_fn, theta, delta=1e-5):
    grad = []
    for i in range(len(theta)):
        theta_perturbed = theta[:]  # Copy to avoid in-place mutation
        theta_perturbed[i] += delta  # Apply small change to one dimension
        grad_i = (loss_fn(theta_perturbed) - loss_fn(theta)) / delta  # Finite difference
        grad.append(grad_i)
    return grad

# Build a gradient descent function with a fixed learning rate alpha
def gradient_descent(loss_fn, alpha):
    def update(theta):
        grads = numerical_gradient(loss_fn, theta)  # Compute gradient at theta
        return [th - alpha * g for th, g in zip(theta, grads)]  # Apply gradient step
    return update

# Recursively apply gradient updates 'revs' times starting from initial theta
def revise(f, revs, theta):
    if revs == 0:
        return theta
    else:
        return revise(f, revs - 1, f(theta))

# High-level wrapper to apply all hyperparameters at once
def with_hypers(hypers, loss_fn, init_theta):
    alpha = hypers.get('alpha')  # Learning rate
    revs = hypers.get('revs')    # Number of gradient descent steps
    return revise(gradient_descent(loss_fn, alpha), revs, init_theta)

In [37]:
# --- Example usage ---

# Sample dataset: xs are inputs, ys are target outputs (y ≈ 2x + 0.1)
# Input vectors (2D features)
xs = [
    [1.0, 2.05],
    [1.0, 3.0],
    [2.0, 2.0],
    [2.0, 3.91],
    [3.0, 6.13],
    [4.0, 8.09],
]

# Target outputs
ys = [13.99, 15.99, 18.0, 22.4, 30.2, 37.94]

# Build a loss function based on the data and model
# loss = l2_loss(line)(xs, ys)
# loss = l2_loss(curve)(xs, ys)
loss = l2_loss(plane)(xs, ys)

# Define hyperparameters
hypers = {
    'alpha': 0.001,    # Learning rate
    'revs': 1000      # Number of training steps
}

# Initial guess for theta
init_theta = [0.0, 0.0, 0.0]

# Train the model using gradient descent
final_theta = with_hypers(hypers, loss, init_theta)

print("Final theta:", final_theta)  # Should be close to [2.0, 0.1]

Final theta: [3.9776771195946576, 2.0495932268171035, 5.786797893779298]


In [40]:
print(final_theta[0]*3 + final_theta[1]*6.13 + final_theta[2])

30.283835732952117
